# Histograms

## Getting Started

Run the cell below to load up the usual datascience tools

In [ ]:
from datascience import *
import numpy as np
%matplotlib inline

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

Then, run the code below to load in the movies data that we were working on in class to the Table named `movies` and then set the format of Adjusted Gross to be numerical and scaled to be measured in Millions of Dollars. The final table is named `millions`

In [ ]:
movies = Table().read_table('https://raw.githubusercontent.com/data-8/textbook/gh-pages/data/top_movies.csv')
movies.set_format([2, 3], NumberFormatter)
millions = movies.select(0).with_column('Adjusted Gross', np.round(movies.column(3)/1e6, 2))
millions

The following code will generate a histogram of the values found in the `Adjusted Gross` column using equally sized bins that are 100 units wide, starting at 300 and ending at 2000. Remember, bins include the lower bound of each bin. For example, the bin from 300 to 400 would include movies that had an `Adjusted Gross` value of 300, but not 400. We'll use the mathematical notation $[300,400)$ to indicate which bound of the interval is inclusive and which is exclusive.

In [ ]:
millions.hist('Adjusted Gross', bins=np.arange(300,2001,100), unit="Million Dollars")

Lastly, this code will create a table that shows each bin and the count of movies that fall into that bin based on the data in the `Adjusted Gross` column.

In [ ]:
bin_counts = millions.bin('Adjusted Gross', bins=np.arange(300,2001,100))
bin_counts.show()

## The Vertical Axis: Density Scale 
The horizontal axis of a histogram is straightforward to read, once we have taken care of details like the ends of the bins. The features of the vertical axis require a little more attention. We will go over them one by one.

Let's start by examining how to calculate the numbers on the vertical axis. If the calculation seems a little strange, have patience – the rest of the handout will explain the reasoning.

### Calculation
The height of each bar is the percent of elements that fall into the corresponding bin, relative to the width of the bin.

In [ ]:
# Creates a Table that counts the number of items in each bin
counts = bin_counts.relabeled('Adjusted Gross count', 'Count')

# Creates a Table that adds a column that contains the perecent
# of movies that fall into each bin
percents = counts.with_column(
    'Percent', (counts.column('Count')/200)*100
    )

# Creates a Table that adds a column that contains the height
# of each bar based on the count in the bin and width of the bin
heights = percents.with_column(
    'Height', percents.column('Percent')/100
    )

# Displays the final table that was created
heights

Go over the numbers on the vertical axis of the histogram the code above generates to check that the column Heights looks correct. The calculations will become clear if we just examine the first row of the table.

Remember that there are 200 movies in the dataset. The $[300, 400)$ bin contains 81 movies. That's 40.5% of all the movies:

$$\text{Percent} = \frac{81}{20} \cdot 100 = 40.5$$

The width of the $[300, 400)$ bin is $400−300 = 100$. So,

$$\text{Height} = \frac{40.5}{400-300} = \frac{40.5}{100} = 0.405$$

The code for calculating the heights used the facts that there are 200 movies in all and that the width of each bin is 100. For datasets of different sizes or for datasets where the bins are not all the same size, the code used would need to be adjusted to take those facts into account.

### Units
The height of the bar is 40.5% divided by 100 million dollars, and so the height is 0.405% per million dollars. This method of drawing histograms creates a vertical axis that is said to be on the density scale. The height of a bar is not the percent of entries in the bin; it is the percent of entries in the bin relative to the amount of space in the bin. That is why the height measures crowdedness or density.

Let's see why this matters.

### Unequal Bins 
An advantage of the histogram over a bar chart is that a histogram can contain bins of unequal width. The code below will take the values in the Millions column and bin them into three uneven categories.

In [ ]:
uneven = make_array(300, 400, 600, 1500)
millions.hist('Adjusted Gross', bins=uneven, unit="Million Dollars")

Here are the counts in the three bins.

In [ ]:
millions.bin('Adjusted Gross', bins=uneven)

Although the ranges $[300, 400)$ and $[400, 600)$ have nearly identical counts, notice that the bar of the former is twice as tall as the latter because it is only half as wide. The density of values in the $[300, 400)$ is twice as much as the density in $[400, 600)$. Histograms help us visualize where on the number line the data are most concentrated, especially when the bins are uneven.

### The Problem with Simply Plotting Counts
It is possible to display counts directly in a chart, using the `normed=False` option of the `hist` method. The resulting chart has the same shape as a histogram when the bins all have equal widths, though the numbers on the vertical axis are different.

In [ ]:
millions.hist('Adjusted Gross', bins=np.arange(300,2001,100), normed=False)

While the count scale is perhaps more natural to interpret than the density scale, the chart becomes highly misleading when bins have different widths. Below, it appears (due to the count scale) that high-grossing movies are quite common, when in fact we have seen that they are relatively rare.

In [ ]:
millions.hist('Adjusted Gross', bins=uneven, normed=False)

Even though the method used is called `hist`, the figure above is **NOT A HISTOGRAM**. It misleadingly exaggerates the proportion of movies grossing at least 600 million dollars. The height of each bar is simply plotted at the number of movies in the bin, without accounting for the difference in the widths of the bins.

The picture becomes even more absurd if the last two bins are combined.

In [ ]:
very_uneven = make_array(300, 400, 1500)
millions.hist('Adjusted Gross', bins=very_uneven, normed=False)

### The Histogram: General Principles and Calculation 
The figure above shows that what the eye perceives as "big" is area, not just height. This observation becomes particularly important when the bins have different widths.

That is why a histogram has two defining properties:

1. The bins are drawn to scale and are contiguous (though some might be empty), because the values on the horizontal axis are numerical.
2. The area of each bar is proportional to the number of entries in the bin.

Property 2 is the key to drawing a histogram, and is usually achieved as follows:

$$\text{area of bar} = \text{percent of entries in bin}$$
 
The calculation of the heights just uses the fact that the bar is a rectangle:

$$\text{area of bar} = \text{height of bar} \cdot \text{width of bin}$$
 
and so

$$\text{height of bar} = \frac{\text{area of bar}}{\text{width of bin}} = \frac{\text{percent of entries in bin}}{\text{width of bin}}$$
 
The units of height are "percent per unit on the horizontal axis."

When drawn using this method, the histogram is said to be drawn on the density scale. On this scale:

* The area of each bar is equal to the percent of data values that are in the corresponding bin.
* The total area of all the bars in the histogram is 100%. Speaking in terms of proportions, we say that the areas of all the bars in a histogram "sum to 1".